#### Importing the libraries needed

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.decomposition import PCA
import math
from sklearn.model_selection import train_test_split

#### Cross sectional time series of firm fundamentals 

In [ ]:
df_predictors = pd.read_csv("../signed_predictors_dl_wide.csv")

#### Cross sectional returns and excess returns from various asset pricing models

In [ ]:
'''df_rf: Risk-free rate, 
df_mkt: Excess return from the Market Model, 
df_ff3: Excess return from the FF3 Model, 
df_ff4: Excess return from the FF4 Model.'''

df_rf = pd.read_csv("../rf.csv")
df_mkt = pd.read_csv("../mkt model.csv")
df_ff3 = pd.read_csv("../ff3.csv")
df_ff4 = pd.read_csv("../ff4.csv")

In [ ]:
df_mkt.columns = ['permno', 'yyyymm', 'n', 'RET_mkt', 'b_mkt', 'alpha', 'ivol', 'tvol', 'R2',
       'exret_mkt']
df_ff3.columns = ['permno', 'yyyymm', 'n', 'RET_ff3', 'alpha', 'b_mkt', 'b_smb', 'b_hml',
       'ivol', 'tvol', 'R2', 'exret_ff3']
df_ff4.columns = ['permno', 'yyyymm', 'n', 'RET_ff4', 'alpha', 'b_mkt', 'b_smb', 'b_hml',
       'b_umd', 'ivol', 'tvol', 'R2', 'exret_ff4']
df_rf.columns = ['yyyymm', 'mktrf', 'smb', 'hml', 'rf', 'umd']

In [ ]:
#Taking data after 2000 into consideration to reduce the amount of noise
df_predictors_after_2000 = df_predictors[df_predictors.yyyymm>200000]

In [ ]:
#preprocessing date column for all dataframes
df_ff4.yyyymm = np.floor(df_ff4.yyyymm/100).astype(int)
df_ff3.yyyymm = np.floor(df_ff3.yyyymm/100).astype(int)
df_mkt.yyyymm = np.floor(df_mkt.yyyymm/100).astype(int)
df_rf.yyyymm = np.floor(df_rf.yyyymm/100).astype(int)

In [ ]:
#filtering dependent variables after 2000's
df_ff4 = df_ff4[df_ff4.yyyymm>200000]
df_ff3 = df_ff3[df_ff3.yyyymm>200000]
df_mkt = df_mkt[df_mkt.yyyymm>200000]
df_rf = df_rf[df_rf.yyyymm>200000]

In [ ]:
#merging dependnet variables to dataset with firm fundamentals
df_predictors_after_2000 = pd.merge(df_predictors_after_2000,df_ff4[['permno','yyyymm','exret_ff4','RET_ff4']],on=['permno','yyyymm'],how='left')

df_predictors_after_2000 = pd.merge(df_predictors_after_2000,df_ff3[['permno','yyyymm','exret_ff3']],on=['permno','yyyymm'],how='left')

df_predictors_after_2000 = pd.merge(df_predictors_after_2000,df_mkt[['permno','yyyymm','exret_mkt']],on=['permno','yyyymm'],how='left')

df_predictors_after_2000 = pd.merge(df_predictors_after_2000,df_rf,on=['yyyymm'],how='left')


In [ ]:
#dropping all records that have null output labels
df_predictors_after_2000 = df_predictors_after_2000.dropna(axis=0, subset=['exret_ff4', 'RET_ff4', 'exret_ff3', 'exret_mkt', 'mktrf', 'smb', 'hml',
       'rf', 'umd'])

In [ ]:
#Forward filling null data with respect to the permno (firm ID) or company identifier
for i in df_predictors_after_2000.columns:
    df_predictors_after_2000[i] = df_predictors_after_2000.groupby('permno')[i].transform(lambda v: v.ffill())

In [ ]:
#deleting records of permno or company identifier that have completely null features
gg = df_predictors_after_2000.groupby("permno")
d = {}
del_permno = []
for x,g in gg:
    l = []
    l = [i for i in g.columns if g[i].isna().mean()==1]
    d[x] = l
    if len(l)>(0.5*df_predictors_after_2000.shape[1]):
        del_permno.append(x)

In [ ]:
df_predictors_after_2000 = df_predictors_after_2000[~(df_predictors_after_2000.permno.isin(del_permno))]

In [ ]:
# Delete records containing 70% or more NaN Values
perc = 70.0
min_count =  int(((100-perc)/100)*df_predictors_after_2000.shape[1] + 1)
df_predictors_after_2000 = df_predictors_after_2000.dropna( axis=0, thresh=min_count)

In [ ]:
#Deleting columns/features that still have more than 50% null values
df_predictors_after_2000.drop([i for i in df_predictors_after_2000.columns if df_predictors_after_2000[i].isna().mean()>0.5],axis = 1, inplace = True)

In [ ]:
#Preprocessing some columns that have the returns in the format "x%", removing "%"" and converting to float
for i in [i for i in df_predictors_after_2000.columns if df_predictors_after_2000[i].dtypes == 'object']:
    df_predictors_after_2000[i] = df_predictors_after_2000[i].str[:-1].astype(float)

In [ ]:
#Imputing the remaining missing values using feature median value.
for i in df_predictors_after_2000.columns:
    median = df_predictors_after_2000[i].median()
    df_predictors_after_2000[i] = df_predictors_after_2000[i].fillna(median)

In [ ]:
y_list = ['RET_ff4','exret_ff4','exret_ff3', 'exret_mkt'] #dependent variables
#independent variables
x_features = [i for i in df_predictors_after_2000.columns if (i not in ['mktrf','smb','hml','umd','permno','exret_ff4', 'RET_ff4', 'exret_ff3', 'exret_mkt','rf'])]

#### Macro-economic variables

In [ ]:
macro_df = pd.read_excel("../Macro_economic_PredictorData2021_Amit_goyal.xlsx",sheet_name='Monthly', header=0)

In [ ]:
macro_df = macro_df[['yyyymm','D12','E12','b/m','tbl','AAA','BAA','lty','ntis','corpr','svar','CRSP_SPvw','CRSP_SPvwx']]


In [ ]:
years = macro_df[macro_df.yyyymm>200000].yyyymm

In [ ]:
macro_df = macro_df[macro_df.yyyymm>200000]

#### Extracting the first 3 principal components of macro variables.

In [ ]:
pca = PCA(n_components=3)
pca_components = pca.fit_transform(macro_df.drop(columns=['yyyymm']))

# Convert PCA components to a dataframe
pca_df = pd.DataFrame(pca_components, columns=[f"PC{i+1}" for i in range(3)])
pca_df['yyyymm'] = years.values
explained_variance = pca.explained_variance_ratio_
print("Total variance explained by the first 3 PCs:",np.round(explained_variance.sum()*100,4),"%")

In [ ]:
#merging the 3 PC's with the predictor data
df_predictors_after_2000 = pd.merge(df_predictors_after_2000,pca_df,on=['yyyymm'],how='left')

In [ ]:
df_predictors_after_2000.head()

#### Interaction terms with the 3 PC's and the original predictor variables.

In [ ]:
interaction_terms = []
for pc in ["PC1", "PC2", "PC3"]:
    for col in x_features[1:]:  # Exclude the Date column
        interaction_terms.append(df_predictors_after_2000[col] * df_predictors_after_2000[pc])

# Combine interaction terms into a single dataframe
interaction_df = pd.concat(interaction_terms, axis=1)
interaction_df.columns = [f"{col}_{pc}" for pc in ["PC1", "PC2", "PC3"] for col in x_features[1:]]

# Step 2: Combine the original 166 columns with the interaction terms
final_df = pd.concat([df_predictors_after_2000[x_features], interaction_df], axis=1)

# Verify the final shape
print("Final dataframe shape:", final_df.shape)  # Should be (number_of_rows, 664)

In [ ]:
thr = 201800
x_train = final_df[final_df.yyyymm<thr][x_features]
y_train = final_df[final_df.yyyymm<thr][y_list]
print(x_train.shape,y_train.shape)
x_test = final_df[final_df.yyyymm>=thr][x_features]
y_test = final_df[final_df.yyyymm>=thr][y_list]
print(x_test.shape,y_test.shape)

In [ ]:
x_train.to_csv("x_train_final.csv",index = False)
y_train.to_csv("y_train_final.csv",index = False)
x_test.to_csv("x_test_final.csv",index = False)
y_test.to_csv("y_test_final.csv",index = False)